In [5]:
# Import necessary libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col, when, regexp_replace, countDistinct, lit, sum, avg, max, min, count,isnan
import os

### Creating a new Spark Session

In [7]:
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk-11'
spark = SparkSession.builder.appName("OlympicsDatasetAnalysis").getOrCreate()
spark

### Loading Data to a DataFrame

Link dataset: https://github.com/KeithGalli/Olympics-Dataset

In [13]:
datafile = "Olympics Dataset .csv"
df = spark.read.csv(datafile,header=True,inferSchema=True)
df.show(5, truncate=False)

+------+------+----------+------------------------+---------------------+----------+---+---------------+-----+-----+-----+
|year  |type  |discipline|event                   |as                   |athlete_id|noc|team           |place|tied |medal|
+------+------+----------+------------------------+---------------------+----------+---+---------------+-----+-----+-----+
|1912.0|Summer|Tennis    |Singles, Men (Olympic)  |Jean-François Blanchy|1         |FRA|NULL           |17.0 |true |NULL |
|1912.0|Summer|Tennis    |Doubles, Men (Olympic)  |Jean-François Blanchy|1         |FRA|Jean Montariol |NULL |false|NULL |
|1920.0|Summer|Tennis    |Singles, Men (Olympic)  |Jean-François Blanchy|1         |FRA|NULL           |32.0 |true |NULL |
|1920.0|Summer|Tennis    |Doubles, Mixed (Olympic)|Jean-François Blanchy|1         |FRA|Jeanne Vaussard|8.0  |true |NULL |
|1920.0|Summer|Tennis    |Doubles, Men (Olympic)  |Jean-François Blanchy|1         |FRA|Jacques Brugnon|4.0  |false|NULL |
+------+------+-

### Data overview

In [16]:
# Number of rows
df.count()

308408

In [32]:
#Describe
df.describe().show(truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------
 summary    | count                                             
 year       | 305807                                            
 type       | 305807                                            
 discipline | 308407                                            
 event      | 308408                                            
 as         | 308408                                            
 athlete_id | 308408                                            
 noc        | 308407                                            
 team       | 121714                                            
 place      | 283193                                            
 medal      | 44139                                             
-RECORD 1-------------------------------------------------------
 summary    | mean                                              
 year       | 1982.239222777765                                 
 type       | NULL       

In [34]:
# Print schema
df.printSchema()

root
 |-- year: double (nullable = true)
 |-- type: string (nullable = true)
 |-- discipline: string (nullable = true)
 |-- event: string (nullable = true)
 |-- as: string (nullable = true)
 |-- athlete_id: integer (nullable = true)
 |-- noc: string (nullable = true)
 |-- team: string (nullable = true)
 |-- place: double (nullable = true)
 |-- tied: boolean (nullable = true)
 |-- medal: string (nullable = true)



### Data cleaning

#### Drop unnecessary columns

In [65]:
df = df.drop('team','type')
df.show(2,truncate=False)

+------+----------+----------------------+---------------------+----------+---+-----+-----+-----+
|year  |discipline|event                 |as                   |athlete_id|noc|place|tied |medal|
+------+----------+----------------------+---------------------+----------+---+-----+-----+-----+
|1912.0|Tennis    |Singles, Men (Olympic)|Jean-François Blanchy|1         |FRA|17.0 |true |NULL |
|1912.0|Tennis    |Doubles, Men (Olympic)|Jean-François Blanchy|1         |FRA|NULL |false|NULL |
+------+----------+----------------------+---------------------+----------+---+-----+-----+-----+
only showing top 2 rows



#### Drop rows with NULL value

In [67]:
df = df.withColumn("tied", col("tied").cast("string"))

In [69]:
# Check missing values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+----------+-----+---+----------+---+-----+----+------+
|year|discipline|event| as|athlete_id|noc|place|tied| medal|
+----+----------+-----+---+----------+---+-----+----+------+
|2601|         1|    0|  0|         0|  1|25215|   0|264269|
+----+----------+-----+---+----------+---+-----+----+------+



In [60]:
df.select("year").distinct().orderBy("year").rdd.flatMap(lambda x: x).collect()

[None,
 1896.0,
 1900.0,
 1904.0,
 1908.0,
 1912.0,
 1920.0,
 1924.0,
 1928.0,
 1932.0,
 1936.0,
 1948.0,
 1952.0,
 1956.0,
 1960.0,
 1964.0,
 1968.0,
 1972.0,
 1976.0,
 1980.0,
 1984.0,
 1988.0,
 1992.0,
 1994.0,
 1996.0,
 1998.0,
 2000.0,
 2002.0,
 2004.0,
 2006.0,
 2008.0,
 2010.0,
 2012.0,
 2014.0,
 2016.0,
 2018.0,
 2020.0,
 2022.0]

Given the data, it's likely that most NULL values in the 'year' column correspond to 1906. The Intercalated Games, a unique event held in 1906 to mark the 10th anniversary of the modern Olympics, were organized by the IOC but are not officially recognized in the Olympic record. As a result, the outcomes of these Games are excluded from the overall Olympic medal count.

In [73]:
df_null_year = df.filter(col("year").isNull())
df_null_year.show(truncate=False)

+----+--------------------------------+-----------------------------+--------------------+----------+---+-----+-----+------+
|year|discipline                      |event                        |as                  |athlete_id|noc|place|tied |medal |
+----+--------------------------------+-----------------------------+--------------------+----------+---+-----+-----+------+
|NULL|Tennis                          |Singles, Men (Intercalated)  |Max Decugis         |12        |FRA|1.0  |false|Gold  |
|NULL|Tennis                          |Doubles, Mixed (Intercalated)|Max Decugis         |12        |FRA|1.0  |false|Gold  |
|NULL|Tennis                          |Doubles, Men (Intercalated)  |Max Decugis         |12        |FRA|1.0  |false|Gold  |
|NULL|Tennis                          |Singles, Men (Intercalated)  |Maurice Germot      |19        |FRA|2.0  |false|Silver|
|NULL|Tennis                          |Doubles, Men (Intercalated)  |Maurice Germot      |19        |FRA|1.0  |false|Gold  |


I will replace all NULL values in the 'year' column with **1906**, provided that the word **(Intercalated)** appears at the end of each record in the 'event' column

In [77]:
df = df.withColumn("year",when((col("year").isNull()) & (col("event").contains("(Intercalated)")), 1906).otherwise(col("year")))

In [79]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+----------+-----+---+----------+---+-----+----+------+
|year|discipline|event| as|athlete_id|noc|place|tied| medal|
+----+----------+-----+---+----------+---+-----+----+------+
| 605|         1|    0|  0|         0|  1|25215|   0|264269|
+----+----------+-----+---+----------+---+-----+----+------+



In [81]:
# Drop rows with NULL values
df = df.dropna(subset=["year", "discipline","noc"])

In [83]:
df.count()

307802

#### Drop duplicated records

In [93]:
df = df.dropDuplicates()
df.count()

307626

### Save as CSV file

In [102]:
import pandas as pd
pandas_df = df.toPandas()
pandas_df.to_csv('Olympics_dataset_cleaned.csv', index=False)